In [2]:
import torch
torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device(type='cuda')

In [116]:
from typing import Literal
from typing import List
# from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasources: str = Field(
        ...,
        description="Given a user question choose which datasources would be most relevant for answering their question",
    )

In [124]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.1:8b",
    temperature=0,
    device="cuda",
    format='json',
)
structured_llm = llm.with_structured_output(RouteQuery.model_json_schema())

system = """Choose which datasources the user's question is closest to.
            Expectred datasources: {datasources}. Use only one the specified datasources."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

router = prompt | structured_llm

In [125]:
question = """What is python?"""
datasources = ['popsci', 'develop', 'design', 'management', 'admin', 'marketing']

router.invoke({"question": question,
               "datasources": datasources})

{'datasources': 'popsci,develop'}

In [ ]:
from ollama import chat
from pydantic import BaseModel

class Country(BaseModel):
  name: str
  capital: str
  languages: list[str]

response = chat(
  messages=[
    {
      'role': 'user',
      'content': 'Tell me about Canada.',
    }
  ],
  model='llama3.1',
  format="",
)

country = Country.model_validate_json(response.message.content)
print(country)

TypeError: Client.chat() got an unexpected keyword argument 'device'

In [ ]:
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
# from pydantic import BaseModel, Field




class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["python_docs", "js_docs", "golang_docs"] = Field(
        description="Given a user question choose which datasource would be most relevant for answering their question",
    )


In [56]:
llm = ChatOllama(
    model="llama3.1",
    temperature=0,
    device="cuda",
    format='json',
)
structured_llm = llm.with_structured_output(RouteQuery)


system = """You are an expert at routing a user question to the appropriate data source.

Based on the programming language the question is referring to, route it to the relevant data source."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

router = prompt | structured_llm

In [58]:
question = """Why doesn't the following code work:


import { ChatPromptTemplate } from "@langchain/core/prompts";


const chatPrompt = ChatPromptTemplate.fromMessages([
  ["human", "speak in {language}"],
]);

const formattedChatPrompt = await chatPrompt.invoke({
  input_language: "french"
});
"""
router.invoke({"question": question})

ValidationError: 1 validation error for RouteQuery
datasource
  unexpected value; permitted: 'python_docs', 'js_docs', 'golang_docs' (type=value_error.const; given=Stack Overflow; permitted=('python_docs', 'js_docs', 'golang_docs'))

In [59]:
from ollama import chat
from pydantic import BaseModel, ValidationError

class Country(BaseModel):
    name: str
    capital: str
    languages: list[str]

response = chat(
    messages=[
        {
            'role': 'user',
            'content': 'Tell me about Canada.',
        }
    ],
    model='llama3.1',
    format='json',
)

# Печатаем ответ для проверки
print(response)

model='llama3.1' created_at='2024-12-13T19:24:56.3722774Z' done=True done_reason='stop' total_duration=830309200 load_duration=62093100 prompt_eval_count=15 prompt_eval_duration=378000000 eval_count=3 eval_duration=378000000 message=Message(role='assistant', content='{ }', images=None, tool_calls=None)


In [41]:
system = """You are an expert at routing a user question to the appropriate data source.

Based on the programming language the question is referring to, route it to the relevant data source."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

question = """Why doesn't the following code work:


import { ChatPromptTemplate } from "@langchain/core/prompts";


const chatPrompt = ChatPromptTemplate.fromMessages([
  ["human", "speak in {language}"],
]);

const formattedChatPrompt = await chatPrompt.invoke({
  input_language: "french"
});
"""

messages = prompt.format_messages(question=question)

ai_msg = llm.invoke(messages)
print(ai_msg.content)

A question about a specific programming language!

Based on the code snippet, I'd say this is likely related to JavaScript and possibly using a framework like LangChain.

I'll route this question to the relevant data source...

 Ah-ha! I've got it!

This question will be routed to the **LangChain Core Prompts** documentation, specifically the section on `ChatPromptTemplate` and its usage in JavaScript.
